In [1]:
#import estential tool
import rdkit
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
import torch_geometric
from torch_geometric.datasets import MoleculeNet
from torch_geometric.data import Data
import networkx as nx
import seaborn as sb
import sklearn
from sklearn.metrics import mean_squared_error

#for trainning
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.nn import Linear
from torch.nn import init
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utill import *
from Model.BASE_model import *
from Model.Less_layer_model import *
from Model.More_layer_model import *
from Model.BASE_model_tanh import *
from Model.Less_layer_model_tanh import *
from Model.More_layer_model_tanh import *
from Model.BASE_model_tanh_pyra import *
from Model.Less_layer_model_tanh_pyra import *
from Model.More_layer_model_tanh_pyra import *
from Model.BASE_model_pyra import *
from Model.Less_layer_model_pyra import *
from Model.More_layer_model_pyra import *
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

seed_value = 42
torch.manual_seed(seed_value)
np.random.seed(seed_value)

# Download dataset
data = MoleculeNet(root=".", name="ESOL")

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Modeling

In [2]:
# setup dataset
data_size = len(data)
NUM_GRAPHS_PER_BATCH = 32
training_set = DataLoader(data[:int(data_size * 0.7)],
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
test_set = DataLoader(data[int(data_size * 0.7):int(data_size * 0.85)],
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
validation_set = DataLoader(data[int(data_size * 0.85):],
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)



In [3]:
#first set
# setup 
loss_fn = torch.nn.MSELoss()

#first model
first_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=32)
PATH_first_model_init = "Save_weight/init_weight/first_model.pth"
# first_model.init_weights()
# torch.save(first_model.state_dict(), PATH_first_model_init)
first_model.load_state_dict(torch.load(PATH_first_model_init))
optimizer_first = torch.optim.Adam(first_model.parameters(), lr=0.0007)

#second model
second_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=64)
PATH_second_model_init = "Save_weight/init_weight/second_model.pth"
# second_model.init_weights()
# torch.save(second_model.state_dict(), PATH_second_model_init)
second_model.load_state_dict(torch.load(PATH_second_model_init))
optimizer_second = torch.optim.Adam(second_model.parameters(), lr=0.0007)

#third model
third_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=128)
PATH_third_model_init = "Save_weight/init_weight/third_model.pth"
# third_model.init_weights()
# torch.save(third_model.state_dict(), PATH_third_model_init)
third_model.load_state_dict(torch.load(PATH_third_model_init))
optimizer_third = torch.optim.Adam(third_model.parameters(), lr=0.0007)

#forth model
forth_model = GCN_More_layer_model(data_num_features=data.num_features, embedding_size=32)
PATH_forth_model_init = "Save_weight/init_weight/forth_model.pth"
# forth_model.init_weights()
# torch.save(forth_model.state_dict(), PATH_forth_model_init)
forth_model.load_state_dict(torch.load(PATH_forth_model_init))
optimizer_forth = torch.optim.Adam(forth_model.parameters(), lr=0.0007)

#fifth model
fifth_model = GCN_More_layer_model(data_num_features=data.num_features, embedding_size=64)
PATH_fifth_model_init = "Save_weight/init_weight/fifth_model.pth"
# fifth_model.init_weights()
# torch.save(fifth_model.state_dict(), PATH_fifth_model_init)
fifth_model.load_state_dict(torch.load(PATH_fifth_model_init))
optimizer_fifth = torch.optim.Adam(fifth_model.parameters(), lr=0.0007)

#sixth model
sixth_model = GCN_More_layer_model(data_num_features=data.num_features, embedding_size=128)
PATH_sixth_model_init = "Save_weight/init_weight/sixth_model.pth"
# sixth_model.init_weights()
# torch.save(sixth_model.state_dict(), PATH_sixth_model_init)
sixth_model.load_state_dict(torch.load(PATH_sixth_model_init))
optimizer_sixth = torch.optim.Adam(sixth_model.parameters(), lr=0.0007)

#seventh model
seventh_model = GCN_Less_layer_model(data_num_features=data.num_features, embedding_size=32)
PATH_seventh_model_init = "Save_weight/init_weight/seventh_model.pth"
# seventh_model.init_weights()
# torch.save(seventh_model.state_dict(), PATH_seventh_model_init)
seventh_model.load_state_dict(torch.load(PATH_seventh_model_init))
optimizer_seventh = torch.optim.Adam(seventh_model.parameters(), lr=0.0007)

#eighth model
eighth_model = GCN_Less_layer_model(data_num_features=data.num_features, embedding_size=64)
PATH_eighth_model_init = "Save_weight/init_weight/eighth_model.pth"
# eighth_model.init_weights()
# torch.save(eighth_model.state_dict(), PATH_eighth_model_init)
eighth_model.load_state_dict(torch.load(PATH_eighth_model_init))
optimizer_eighth = torch.optim.Adam(eighth_model.parameters(), lr=0.0007)

#ninth model
ninth_model = GCN_Less_layer_model(data_num_features=data.num_features, embedding_size=128)
PATH_ninth_model_init = "Save_weight/init_weight/ninth_model.pth"
# ninth_model.init_weights()
# torch.save(ninth_model.state_dict(), PATH_ninth_model_init)
ninth_model.load_state_dict(torch.load(PATH_ninth_model_init))
optimizer_ninth = torch.optim.Adam(ninth_model.parameters(), lr=0.0007)


In [4]:
# # first_model trainning
# loss_fn = torch.nn.MSELoss()
# first_model = GCN_BASE_model_tanh(data_num_features=data.num_features, embedding_size=32)
# first_model.load_state_dict(torch.load(PATH_first_model_init))
# optimizer_first = torch.optim.Adam(first_model.parameters(), lr=0.0007)
# loss_train_track_first, loss_valid_track_first, stop_at_epoch_first = train(model=first_model, optimizer=optimizer_first, loss_fn=loss_fn,
#                                                                             epochs=5, is_early_stop=False, show_result_at=1,
#                                                                             device=device, validation_set=validation_set, training_set=training_set)


In [5]:
#first model
first_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=32).to(device)
PATH_first_model_init = "Save_weight/init_weight/first_model.pth"
# first_model.init_weights()
# torch.save(first_model.state_dict(), PATH_first_model_init)
first_model.load_state_dict(torch.load(PATH_first_model_init))
optimizer_first = torch.optim.Adam(first_model.parameters(), lr=0.0007)

In [6]:
#first section train
epoch = 30
save_at = 5
continue_at = 0
# first_model trainning
PATH_to_esol_drive = "Debug/"
first_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=32).to(device)
first_model.load_state_dict(torch.load(PATH_first_model_init))
# first_model.load_state_dict(torch.load("Debug/first_model_e5.pth"))

optimizer_first = torch.optim.Adam(first_model.parameters(), lr=0.0007)
# optimizer_first.load_state_dict(torch.load("Debug/first_modeloptimizer_e5.pth"))
loss_train_track_first, loss_valid_track_first, stop_at_epoch_first = train(model=first_model, optimizer=optimizer_first, loss_fn=loss_fn,
                                                                            epochs=epoch, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set,
                                                                            save_path = PATH_to_esol_drive,save_every=save_at,is_auto_save=True,model_name="first_model",
                                                                            continue_epoch=continue_at)


at epoch : 1 train_loss = 1310.6520776367188  valid_loss = 433.23113505045575
at epoch : 2 train_loss = 212.87633819580077  valid_loss = 212.7928899129232
at epoch : 3 train_loss = 145.24710052490235  valid_loss = 145.94114303588867
at epoch : 4 train_loss = 121.67669036865234  valid_loss = 124.27682113647461
at epoch : 5 train_loss = 107.7506118774414  valid_loss = 110.82983271280925
at epoch : 6 train_loss = 95.63193283081054  valid_loss = 97.61493873596191
at epoch : 7 train_loss = 83.68597747802734  valid_loss = 84.0013821919759
at epoch : 8 train_loss = 72.77913497924804  valid_loss = 72.19316418965657
at epoch : 9 train_loss = 64.29444137573242  valid_loss = 64.05376402537028
at epoch : 10 train_loss = 58.01697982788086  valid_loss = 57.81423314412435
at epoch : 11 train_loss = 52.866600036621094  valid_loss = 52.585845947265625
at epoch : 12 train_loss = 48.46156234741211  valid_loss = 48.0396925608317
at epoch : 13 train_loss = 44.566526641845705  valid_loss = 44.0296246210734


In [7]:
#first section train
epoch = 30
save_at = 0
continue_at = 0
# first_model trainning
PATH_to_esol_drive = "Debug/"
first_model_test = GCN_BASE_model(data_num_features=data.num_features, embedding_size=32).to(device)
first_model_test.load_state_dict(torch.load(PATH_first_model_init))
# first_model_test.load_state_dict(torch.load("Debug/first_model_e30.pth"))

optimizer_first_test = torch.optim.Adam(first_model_test.parameters(), lr=0.0007)
# optimizer_first_test.load_state_dict(torch.load("Debug/first_modeloptimizer_e30.pth"))
loss_train_track_first, loss_valid_track_first, stop_at_epoch_first = train(model=first_model_test, optimizer=optimizer_first_test, loss_fn=loss_fn,
                                                                            epochs=epoch, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set,
                                                                            save_path = PATH_to_esol_drive,save_every=save_at,is_auto_save=True,model_name="first_model",
                                                                            continue_epoch=continue_at)


at epoch : 1 train_loss = 1310.6520776367188  valid_loss = 433.23113505045575
at epoch : 2 train_loss = 212.87633819580077  valid_loss = 212.7928899129232
at epoch : 3 train_loss = 145.24710052490235  valid_loss = 145.94114303588867
at epoch : 4 train_loss = 121.67669036865234  valid_loss = 124.27682113647461
at epoch : 5 train_loss = 107.7506118774414  valid_loss = 110.82983271280925
at epoch : 6 train_loss = 95.63193283081054  valid_loss = 97.61493873596191
at epoch : 7 train_loss = 83.68597747802734  valid_loss = 84.0013821919759
at epoch : 8 train_loss = 72.77913497924804  valid_loss = 72.19316418965657
at epoch : 9 train_loss = 64.29444137573242  valid_loss = 64.05376402537028
at epoch : 10 train_loss = 58.01697982788086  valid_loss = 57.81423314412435
at epoch : 11 train_loss = 52.866600036621094  valid_loss = 52.585845947265625
at epoch : 12 train_loss = 48.46156234741211  valid_loss = 48.0396925608317
at epoch : 13 train_loss = 44.566526641845705  valid_loss = 44.0296246210734


In [8]:
print(test(model=first_model,loss_fn=loss_fn,test_set=test_set,device=device))
print(test(model=first_model_test,loss_fn=loss_fn,test_set=test_set,device=device))

17.816683451334637
17.816683451334637


In [ ]:
17.816683451334637
17.62319215138753

In [69]:
for i in training_set:
    print(i)

DataBatch(x=[379, 9], edge_index=[2, 774], edge_attr=[774, 3], smiles=[32], y=[32, 1], batch=[379], ptr=[33])
DataBatch(x=[459, 9], edge_index=[2, 970], edge_attr=[970, 3], smiles=[32], y=[32, 1], batch=[459], ptr=[33])
DataBatch(x=[421, 9], edge_index=[2, 858], edge_attr=[858, 3], smiles=[32], y=[32, 1], batch=[421], ptr=[33])
DataBatch(x=[451, 9], edge_index=[2, 922], edge_attr=[922, 3], smiles=[32], y=[32, 1], batch=[451], ptr=[33])
DataBatch(x=[444, 9], edge_index=[2, 934], edge_attr=[934, 3], smiles=[32], y=[32, 1], batch=[444], ptr=[33])
DataBatch(x=[411, 9], edge_index=[2, 846], edge_attr=[846, 3], smiles=[32], y=[32, 1], batch=[411], ptr=[33])
DataBatch(x=[369, 9], edge_index=[2, 754], edge_attr=[754, 3], smiles=[32], y=[32, 1], batch=[369], ptr=[33])
DataBatch(x=[467, 9], edge_index=[2, 962], edge_attr=[962, 3], smiles=[32], y=[32, 1], batch=[467], ptr=[33])
DataBatch(x=[444, 9], edge_index=[2, 936], edge_attr=[936, 3], smiles=[32], y=[32, 1], batch=[444], ptr=[33])
DataBatch(

In [71]:
for i in test_set :
    print(i)

DataBatch(x=[447, 9], edge_index=[2, 932], edge_attr=[932, 3], smiles=[32], y=[32, 1], batch=[447], ptr=[33])
DataBatch(x=[426, 9], edge_index=[2, 886], edge_attr=[886, 3], smiles=[32], y=[32, 1], batch=[426], ptr=[33])
DataBatch(x=[467, 9], edge_index=[2, 972], edge_attr=[972, 3], smiles=[32], y=[32, 1], batch=[467], ptr=[33])
DataBatch(x=[394, 9], edge_index=[2, 794], edge_attr=[794, 3], smiles=[32], y=[32, 1], batch=[394], ptr=[33])
DataBatch(x=[404, 9], edge_index=[2, 824], edge_attr=[824, 3], smiles=[32], y=[32, 1], batch=[404], ptr=[33])
DataBatch(x=[111, 9], edge_index=[2, 222], edge_attr=[222, 3], smiles=[9], y=[9, 1], batch=[111], ptr=[10])


In [76]:
for i in test_set :
    print(i)

DataBatch(x=[408, 9], edge_index=[2, 852], edge_attr=[852, 3], smiles=[32], y=[32, 1], batch=[408], ptr=[33])
DataBatch(x=[409, 9], edge_index=[2, 840], edge_attr=[840, 3], smiles=[32], y=[32, 1], batch=[409], ptr=[33])
DataBatch(x=[455, 9], edge_index=[2, 934], edge_attr=[934, 3], smiles=[32], y=[32, 1], batch=[455], ptr=[33])
DataBatch(x=[427, 9], edge_index=[2, 866], edge_attr=[866, 3], smiles=[32], y=[32, 1], batch=[427], ptr=[33])
DataBatch(x=[435, 9], edge_index=[2, 902], edge_attr=[902, 3], smiles=[32], y=[32, 1], batch=[435], ptr=[33])
DataBatch(x=[115, 9], edge_index=[2, 236], edge_attr=[236, 3], smiles=[9], y=[9, 1], batch=[115], ptr=[10])


In [41]:
14.86410919825236
15.093841870625814

15.093841870625814

In [38]:
test_loading = GCN_BASE_model(data_num_features=data.num_features, embedding_size=32).to(device)
test_loading.load_state_dict(torch.load("Debug/first_model_e5.pth"))
for i in test_loading.parameters():
    print(i)
# for param_tensor in test_loading.state_dict():
    # print(param_tensor, "\t", test_loading.state_dict()[param_tensor].size())

Parameter containing:
tensor([ 0.0126, -0.0116,  0.0000,  0.0095,  0.0170,  0.0000,  0.0000,  0.0122,
         0.0000, -0.0073,  0.0087,  0.0000,  0.0000,  0.0112,  0.0000, -0.0126,
         0.0303,  0.0030,  0.0000,  0.0000, -0.0145,  0.0000,  0.0167,  0.0000,
         0.0000,  0.0056,  0.0112,  0.0112,  0.0036, -0.0076,  0.0000, -0.0100],
       requires_grad=True)
Parameter containing:
tensor([[ 9.9713e-01,  1.3733e+00, -3.1562e-01, -5.2806e-01,  5.4191e-01,
         -1.1346e+00,  4.8438e-01, -7.0008e-01,  4.5105e-01],
        [ 5.5859e-01,  2.4628e-01,  2.8755e+00, -5.6728e-01, -4.6676e-03,
         -3.9583e-01,  3.9306e-01,  1.5500e-01, -2.7219e+00],
        [-1.2835e-01, -4.4340e-01, -1.7999e+00, -4.8732e-01, -3.6020e-01,
         -8.5379e-01,  1.3559e+00, -6.2743e-01,  4.7283e-01],
        [-6.7590e-01,  1.4168e+00,  8.6890e-01,  8.9256e-01, -8.8550e-01,
         -1.3402e+00,  4.4579e-02, -4.4704e-01,  1.3631e-01],
        [ 1.6322e+00, -4.2968e-01, -1.2002e-01,  3.1908e-01, -9.

In [36]:
# Print optimizer's state_dict
test_op  = torch.optim.Adam(test_loading.parameters(), lr=0.0007)
test_op.load_state_dict(torch.load("Debug/first_modeloptimizer_e5.pth"))

print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Optimizer's state_dict:
state 	 {0: {'step': tensor(250.), 'exp_avg': tensor([ 0.3994, -1.5455,  0.0000,  1.8998, -0.3678,  0.0000,  0.0000, -0.5411,
         0.0000,  0.4349,  0.5033,  0.0000,  0.0000,  5.7753,  0.0000,  1.4242,
        -0.4399,  1.2948,  0.0000,  0.0000, -0.4933,  0.0000, -0.3783,  0.0000,
         0.0000,  0.2415, -0.4265, -0.7635, -2.0682,  0.5678,  0.0000,  3.3379]), 'exp_avg_sq': tensor([3.3387e+02, 1.0446e+04, 0.0000e+00, 6.7252e+02, 1.2933e+01, 0.0000e+00,
        0.0000e+00, 9.2839e+02, 0.0000e+00, 7.1529e+03, 3.6696e+02, 0.0000e+00,
        0.0000e+00, 4.0669e+03, 0.0000e+00, 1.4374e+02, 6.2673e-01, 2.4534e+01,
        0.0000e+00, 0.0000e+00, 2.9381e+03, 0.0000e+00, 6.4072e+00, 0.0000e+00,
        0.0000e+00, 2.3226e+02, 1.5215e+03, 1.4339e+03, 3.8165e+01, 8.4634e+02,
        0.0000e+00, 8.6629e+01])}, 1: {'step': tensor(250.), 'exp_avg': tensor([[ 1.7100e+00,  0.0000e+00,  4.3469e+00,  1.7947e+00,  1.9313e-01,
          0.0000e+00,  2.8990e+00, -8.1811e-01, 

In [33]:
for i in first_model.parameters():
    print(i)

Parameter containing:
tensor([ 0.0126, -0.0116,  0.0000,  0.0095,  0.0170,  0.0000,  0.0000,  0.0122,
         0.0000, -0.0073,  0.0087,  0.0000,  0.0000,  0.0112,  0.0000, -0.0126,
         0.0303,  0.0030,  0.0000,  0.0000, -0.0145,  0.0000,  0.0167,  0.0000,
         0.0000,  0.0056,  0.0112,  0.0112,  0.0036, -0.0076,  0.0000, -0.0100],
       requires_grad=True)
Parameter containing:
tensor([[ 9.9713e-01,  1.3733e+00, -3.1562e-01, -5.2806e-01,  5.4191e-01,
         -1.1346e+00,  4.8438e-01, -7.0008e-01,  4.5105e-01],
        [ 5.5859e-01,  2.4628e-01,  2.8755e+00, -5.6728e-01, -4.6676e-03,
         -3.9583e-01,  3.9306e-01,  1.5500e-01, -2.7219e+00],
        [-1.2835e-01, -4.4340e-01, -1.7999e+00, -4.8732e-01, -3.6020e-01,
         -8.5379e-01,  1.3559e+00, -6.2743e-01,  4.7283e-01],
        [-6.7590e-01,  1.4168e+00,  8.6890e-01,  8.9256e-01, -8.8550e-01,
         -1.3402e+00,  4.4579e-02, -4.4704e-01,  1.3631e-01],
        [ 1.6322e+00, -4.2968e-01, -1.2002e-01,  3.1908e-01, -9.

In [25]:
model = first_model
optimizer = optimizer_first

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

Model's state_dict:
initial_conv.bias 	 torch.Size([32])
initial_conv.lin.weight 	 torch.Size([32, 9])
conv1.bias 	 torch.Size([32])
conv1.lin.weight 	 torch.Size([32, 32])
conv2.bias 	 torch.Size([32])
conv2.lin.weight 	 torch.Size([32, 32])
out.weight 	 torch.Size([1, 64])
out.bias 	 torch.Size([1])
Optimizer's state_dict:
state 	 {0: {'step': tensor(250.), 'exp_avg': tensor([ 0.3994, -1.5455,  0.0000,  1.8998, -0.3678,  0.0000,  0.0000, -0.5411,
         0.0000,  0.4349,  0.5033,  0.0000,  0.0000,  5.7753,  0.0000,  1.4242,
        -0.4399,  1.2948,  0.0000,  0.0000, -0.4933,  0.0000, -0.3783,  0.0000,
         0.0000,  0.2415, -0.4265, -0.7635, -2.0682,  0.5678,  0.0000,  3.3379]), 'exp_avg_sq': tensor([3.3387e+02, 1.0446e+04, 0.0000e+00, 6.7252e+02, 1.2933e+01, 0.0000e+00,
        0.0000e+00, 9.2839e+02, 0.0000e+00, 7.1529e+03, 3.6696e+02, 0.0000e+00,
        0.0000e+00, 4.0669e+03, 0.0000e+00, 1.4374e+02, 6.2673e-01, 2.4534e+01,
        0.0000e+00, 0.0000e+00, 2.9381e+03, 0.0000e

In [3]:
print(first_model)
print(second_model)
print(third_model)
print(forth_model)
print(fifth_model)
print(sixth_model)
print(seventh_model)
print(eighth_model)
print(ninth_model)

GCN_BASE_model(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
GCN_BASE_model(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
GCN_BASE_model(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(128, 128)
  (conv2): GCNConv(128, 128)
  (out): Linear(in_features=256, out_features=1, bias=True)
)
GCN_More_layer_model(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
GCN_More_layer_model(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
GCN_More_layer_model(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(128, 128)
  (conv2): GCNConv(12

In [4]:
#second set
#first model
m11 = GCN_BASE_model_tanh(data_num_features=data.num_features, embedding_size=32)
PATH_m11_init = "Save_weight/init_weight/m11.pth"
# m11.init_weights()
# torch.save(m11.state_dict(), PATH_m11_init)
m11.load_state_dict(torch.load(PATH_m11_init))
optimizer_m11 = torch.optim.Adam(m11.parameters(), lr=0.0007)

#second model
m12 = GCN_BASE_model_tanh(data_num_features=data.num_features, embedding_size=64)
PATH_m12_init = "Save_weight/init_weight/m12.pth"
# m12.init_weights()
# torch.save(m12.state_dict(), PATH_m12_init)
m12.load_state_dict(torch.load(PATH_m12_init))
optimizer_m12 = torch.optim.Adam(m12.parameters(), lr=0.0007)

#third model
m13 = GCN_BASE_model_tanh(data_num_features=data.num_features, embedding_size=128)
PATH_m13_init = "Save_weight/init_weight/m13.pth"
# m13.init_weights()
# torch.save(m13.state_dict(), PATH_m13_init)
m13.load_state_dict(torch.load(PATH_m13_init))
optimizer_m13 = torch.optim.Adam(m13.parameters(), lr=0.0007)

#forth model
m14 = GCN_More_layer_model_tanh(data_num_features=data.num_features, embedding_size=32)
PATH_m14_init = "Save_weight/init_weight/m14.pth"
# m14.init_weights()
# torch.save(m14.state_dict(), PATH_m14_init)
m14.load_state_dict(torch.load(PATH_m14_init))
optimizer_m14= torch.optim.Adam(m14.parameters(), lr=0.0007)

#fifth model
m15 = GCN_More_layer_model_tanh(data_num_features=data.num_features, embedding_size=64)
PATH_m15_init = "Save_weight/init_weight/m15.pth"
# m15.init_weights()
# torch.save(m15.state_dict(), PATH_m15_init)
m15.load_state_dict(torch.load(PATH_m15_init))
optimizer_m15 = torch.optim.Adam(m15.parameters(), lr=0.0007)

#sixth model
m16 = GCN_More_layer_model_tanh(data_num_features=data.num_features, embedding_size=128)
PATH_m16_init = "Save_weight/init_weight/m16.pth"
# m16.init_weights()
# torch.save(m16.state_dict(), PATH_m16_init)
m16.load_state_dict(torch.load(PATH_m16_init))
optimizer_m16 = torch.optim.Adam(m16.parameters(), lr=0.0007)

#seventh model
m17 = GCN_Less_layer_model_tanh(data_num_features=data.num_features, embedding_size=32)
PATH_m17_init = "Save_weight/init_weight/m17.pth"
# m17.init_weights()
# torch.save(m17.state_dict(), PATH_m17_init)
m17.load_state_dict(torch.load(PATH_m17_init))
optimizer_m17 = torch.optim.Adam(m17.parameters(), lr=0.0007)

#eighth model
m18 = GCN_Less_layer_model_tanh(data_num_features=data.num_features, embedding_size=64)
PATH_m18_init = "Save_weight/init_weight/m18.pth"
# m18.init_weights()
# torch.save(m18.state_dict(), PATH_m18_init)
m18.load_state_dict(torch.load(PATH_m18_init))
optimizer_m18 = torch.optim.Adam(m18.parameters(), lr=0.0007)

#ninth model
m19 = GCN_Less_layer_model_tanh(data_num_features=data.num_features, embedding_size=128)
PATH_m19_init = "Save_weight/init_weight/m19.pth"
# m19.init_weights()
# torch.save(m19.state_dict(), PATH_m19_init)
m19.load_state_dict(torch.load(PATH_m19_init))
optimizer_m19 = torch.optim.Adam(m19.parameters(), lr=0.0007)

In [5]:
print(m11)
print(m12)
print(m13)
print(m14)
print(m15)
print(m16)
print(m17)
print(m18)
print(m19)

GCN_BASE_model_tanh(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
GCN_BASE_model_tanh(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
GCN_BASE_model_tanh(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(128, 128)
  (conv2): GCNConv(128, 128)
  (out): Linear(in_features=256, out_features=1, bias=True)
)
GCN_More_layer_model_tanh(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
GCN_More_layer_model_tanh(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (out): Linear(in_features=128, out_features=1, bias=True)
)
GCN_More_layer_model_tanh(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(1

In [8]:
#third set
#first model
m21 = GCN_BASE_model_pyra(data_num_features=data.num_features, embedding_size=32)
PATH_m21_init = "Save_weight/init_weight/m21.pth"
# m21.init_weights()
# torch.save(m21.state_dict(), PATH_m21_init)
m21.load_state_dict(torch.load(PATH_m21_init))
optimizer_m21 = torch.optim.Adam(m21.parameters(), lr=0.0007)

#second model
m22 = GCN_BASE_model_pyra(data_num_features=data.num_features, embedding_size=64)
PATH_m22_init = "Save_weight/init_weight/m22.pth"
# m22.init_weights()
# torch.save(m22.state_dict(), PATH_m22_init)
m22.load_state_dict(torch.load(PATH_m22_init))
optimizer_m22 = torch.optim.Adam(m22.parameters(), lr=0.0007)

#third model
m23 = GCN_BASE_model_pyra(data_num_features=data.num_features, embedding_size=128)
PATH_m23_init = "Save_weight/init_weight/m23.pth"
# m23.init_weights()
# torch.save(m23.state_dict(), PATH_m23_init)
m23.load_state_dict(torch.load(PATH_m23_init))
optimizer_m23 = torch.optim.Adam(m23.parameters(), lr=0.0007)

#forth model
m24 = GCN_More_layer_model_pyra(data_num_features=data.num_features, embedding_size=32)
PATH_m24_init = "Save_weight/init_weight/m24.pth"
# m24.init_weights()
# torch.save(m24.state_dict(), PATH_m24_init)
m24.load_state_dict(torch.load(PATH_m24_init))
optimizer_m24= torch.optim.Adam(m24.parameters(), lr=0.0007)

#fifth model
m25 = GCN_More_layer_model_pyra(data_num_features=data.num_features, embedding_size=64)
PATH_m25_init = "Save_weight/init_weight/m25.pth"
# m25.init_weights()
# torch.save(m25.state_dict(), PATH_m25_init)
m25.load_state_dict(torch.load(PATH_m25_init))
optimizer_m25 = torch.optim.Adam(m25.parameters(), lr=0.0007)

#sixth model
m26 = GCN_More_layer_model_pyra(data_num_features=data.num_features, embedding_size=128)
PATH_m26_init = "Save_weight/init_weight/m26.pth"
# m26.init_weights()
# torch.save(m26.state_dict(), PATH_m26_init)
m26.load_state_dict(torch.load(PATH_m26_init))
optimizer_m26 = torch.optim.Adam(m26.parameters(), lr=0.0007)

#seventh model
m27 = GCN_Less_layer_model_pyra(data_num_features=data.num_features, embedding_size=32)
PATH_m27_init = "Save_weight/init_weight/m27.pth"
# m27.init_weights()
# torch.save(m27.state_dict(), PATH_m27_init)
m27.load_state_dict(torch.load(PATH_m27_init))
optimizer_m27 = torch.optim.Adam(m27.parameters(), lr=0.0007)

#eighth model
m28 = GCN_Less_layer_model_pyra(data_num_features=data.num_features, embedding_size=64)
PATH_m28_init = "Save_weight/init_weight/m28.pth"
# m28.init_weights()
# torch.save(m28.state_dict(), PATH_m28_init)
m28.load_state_dict(torch.load(PATH_m28_init))
optimizer_m28 = torch.optim.Adam(m28.parameters(), lr=0.0007)

#ninth model
m29 = GCN_Less_layer_model_pyra(data_num_features=data.num_features, embedding_size=128)
PATH_m29_init = "Save_weight/init_weight/m29.pth"
# m29.init_weights()
# torch.save(m29.state_dict(), PATH_m29_init)
m29.load_state_dict(torch.load(PATH_m29_init))
optimizer_m29 = torch.optim.Adam(m29.parameters(), lr=0.0007)

In [9]:
print(m21)
print(m22)
print(m23)
print(m24)
print(m25)
print(m26)
print(m27)
print(m28)
print(m29)

GCN_BASE_model_pyra(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 16)
  (conv2): GCNConv(16, 8)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
GCN_BASE_model_pyra(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 32)
  (conv2): GCNConv(32, 16)
  (out): Linear(in_features=32, out_features=1, bias=True)
)
GCN_BASE_model_pyra(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(128, 64)
  (conv2): GCNConv(64, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
GCN_More_layer_model_pyra(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 16)
  (conv2): GCNConv(16, 8)
  (conv3): GCNConv(8, 4)
  (out): Linear(in_features=8, out_features=1, bias=True)
)
GCN_More_layer_model_pyra(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 32)
  (conv2): GCNConv(32, 16)
  (conv3): GCNConv(16, 8)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
GCN_More_layer_model_pyra(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(128, 64)
  (c

In [17]:
#forth set
#first model
m31 = GCN_BASE_model_tanh_pyra(data_num_features=data.num_features, embedding_size=32)
PATH_m31_init = "Save_weight/init_weight/m31.pth"
# m31.init_weights()
# torch.save(m31.state_dict(), PATH_m31_init)
m31.load_state_dict(torch.load(PATH_m31_init))
optimizer_m31 = torch.optim.Adam(m31.parameters(), lr=0.0007)

#second model
m32 = GCN_BASE_model_tanh_pyra(data_num_features=data.num_features, embedding_size=64)
PATH_m32_init = "Save_weight/init_weight/m32.pth"
# m32.init_weights()
# torch.save(m32.state_dict(), PATH_m32_init)
m32.load_state_dict(torch.load(PATH_m32_init))
optimizer_m32 = torch.optim.Adam(m32.parameters(), lr=0.0007)

#third model
m33 = GCN_BASE_model_tanh_pyra(data_num_features=data.num_features, embedding_size=128)
PATH_m33_init = "Save_weight/init_weight/m33.pth"
# m33.init_weights()
# torch.save(m33.state_dict(), PATH_m33_init)
m33.load_state_dict(torch.load(PATH_m33_init))
optimizer_m33 = torch.optim.Adam(m33.parameters(), lr=0.0007)

#forth model
m34 = GCN_More_layer_model_tanh_pyra(data_num_features=data.num_features, embedding_size=32)
PATH_m34_init = "Save_weight/init_weight/m34.pth"
# m34.init_weights()
# torch.save(m34.state_dict(), PATH_m34_init)
m34.load_state_dict(torch.load(PATH_m34_init))
optimizer_m34= torch.optim.Adam(m34.parameters(), lr=0.0007)

#fifth model
m35 = GCN_More_layer_model_tanh_pyra(data_num_features=data.num_features, embedding_size=64)
PATH_m35_init = "Save_weight/init_weight/m35.pth"
# m35.init_weights()
# torch.save(m35.state_dict(), PATH_m35_init)
m35.load_state_dict(torch.load(PATH_m35_init))
optimizer_m35 = torch.optim.Adam(m35.parameters(), lr=0.0007)

#sixth model
m36 = GCN_More_layer_model_tanh_pyra(data_num_features=data.num_features, embedding_size=128)
PATH_m36_init = "Save_weight/init_weight/m36.pth"
# m36.init_weights()
# torch.save(m36.state_dict(), PATH_m36_init)
m36.load_state_dict(torch.load(PATH_m36_init))
optimizer_m36 = torch.optim.Adam(m36.parameters(), lr=0.0007)

#seventh model
m37 = GCN_Less_layer_model_tanh_pyra(data_num_features=data.num_features, embedding_size=32)
PATH_m37_init = "Save_weight/init_weight/m37.pth"
# m37.init_weights()
# torch.save(m37.state_dict(), PATH_m37_init)
m37.load_state_dict(torch.load(PATH_m37_init))
optimizer_m37 = torch.optim.Adam(m37.parameters(), lr=0.0007)

#eighth model
m38 = GCN_Less_layer_model_tanh_pyra(data_num_features=data.num_features, embedding_size=64)
PATH_m38_init = "Save_weight/init_weight/m38.pth"
# m38.init_weights()
# torch.save(m38.state_dict(), PATH_m38_init)
m38.load_state_dict(torch.load(PATH_m38_init))
optimizer_m38 = torch.optim.Adam(m38.parameters(), lr=0.0007)

#ninth model
m39 = GCN_Less_layer_model_tanh_pyra(data_num_features=data.num_features, embedding_size=128)
PATH_m39_init = "Save_weight/init_weight/m39.pth"
# m39.init_weights()
# torch.save(m39.state_dict(), PATH_m39_init)
m39.load_state_dict(torch.load(PATH_m39_init))
optimizer_m39 = torch.optim.Adam(m39.parameters(), lr=0.0007)

In [18]:
print(m31)
print(m32)
print(m33)
print(m34)
print(m35)
print(m36)
print(m37)
print(m38)
print(m39)

GCN_BASE_model_tanh_pyra(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 16)
  (conv2): GCNConv(16, 8)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
GCN_BASE_model_tanh_pyra(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 32)
  (conv2): GCNConv(32, 16)
  (out): Linear(in_features=32, out_features=1, bias=True)
)
GCN_BASE_model_tanh_pyra(
  (initial_conv): GCNConv(9, 128)
  (conv1): GCNConv(128, 64)
  (conv2): GCNConv(64, 32)
  (out): Linear(in_features=64, out_features=1, bias=True)
)
GCN_More_layer_model_tanh_pyra(
  (initial_conv): GCNConv(9, 32)
  (conv1): GCNConv(32, 16)
  (conv2): GCNConv(16, 8)
  (conv3): GCNConv(8, 4)
  (out): Linear(in_features=8, out_features=1, bias=True)
)
GCN_More_layer_model_tanh_pyra(
  (initial_conv): GCNConv(9, 64)
  (conv1): GCNConv(64, 32)
  (conv2): GCNConv(32, 16)
  (conv3): GCNConv(16, 8)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
GCN_More_layer_model_tanh_pyra(
  (initial_conv): GCNConv(9, 128)
  

# Setup

In [4]:
# setup dataset
data_size = len(data)
NUM_GRAPHS_PER_BATCH = 32
training_set = DataLoader(data[:int(data_size * 0.7)],
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_set = DataLoader(data[int(data_size * 0.7):int(data_size * 0.85)],
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
validation_set = DataLoader(data[int(data_size * 0.85):],
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)



In [1]:
# first_model trainning
first_model = GCN_BASE_model_tanh(data_num_features=data.num_features, embedding_size=32)
first_model.load_state_dict(torch.load(PATH_first_model_init))
optimizer_first = torch.optim.Adam(first_model.parameters(), lr=0.0007)
loss_train_track_first, loss_valid_track_first, stop_at_epoch_first = train(model=first_model, optimizer=optimizer_first, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


NameError: name 'GCN_BASE_model_tanh' is not defined

In [9]:
# m = torch.nn.Tanh()
# input = torch.randn(506,32)
# print(input.shape)
# m(input)

torch.Size([506, 32])


tensor([[ 0.6300,  0.0249,  0.1932,  ..., -0.8402,  0.3456, -0.7654],
        [-0.6404, -0.4978, -0.2227,  ..., -0.1386,  0.2953,  0.1464],
        [ 0.8533,  0.6839,  0.6223,  ...,  0.5807,  0.5475,  0.2509],
        ...,
        [ 0.3823,  0.0369,  0.7951,  ...,  0.9008,  0.5397, -0.0076],
        [ 0.1935,  0.6556,  0.9537,  ...,  0.8938,  0.8869,  0.5325],
        [ 0.9676, -0.7779, -0.7086,  ...,  0.0625, -0.7975,  0.6519]])

# Training

In [12]:
# first_model trainning
first_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=32)
first_model.load_state_dict(torch.load(PATH_first_model_init))
optimizer_first = torch.optim.Adam(first_model.parameters(), lr=0.0007)
loss_train_track_first, loss_valid_track_first, stop_at_epoch_first = train(model=first_model, optimizer=optimizer_first, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 1223.36765625  valid_loss = 444.2113978068034
at epoch : 2 train_loss = 218.0674462890625  valid_loss = 180.34984334309897
at epoch : 3 train_loss = 140.02480194091797  valid_loss = 153.50049463907877
at epoch : 4 train_loss = 119.71230773925781  valid_loss = 131.25205357869467
at epoch : 5 train_loss = 106.41873245239258  valid_loss = 114.71997578938802


In [52]:
# second_model trainning
second_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=64)
second_model.load_state_dict(torch.load(PATH_second_model_init))
optimizer_second = torch.optim.Adam(second_model.parameters(), lr=0.0007)
loss_train_track_second, loss_valid_track_second, stop_at_epoch_second = train(model=second_model, optimizer=optimizer_second, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


case valid better case 1
case valid better case 2
case valid better case 3
case valid better case 1


[253.21293131510416, 219.31498962402344, 146.00571568806967]

In [13]:
# third_model trainning
third_model = GCN_BASE_model(data_num_features=data.num_features, embedding_size=128)
third_model.load_state_dict(torch.load(PATH_third_model_init))
optimizer_third = torch.optim.Adam(third_model.parameters(), lr=0.0007)
loss_train_track_third, loss_valid_track_third, stop_at_epoch_third = train(model=third_model, optimizer=optimizer_third, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 14010.541247558594  valid_loss = 7101.525227864583
at epoch : 2 train_loss = 3068.4201904296874  valid_loss = 1198.3312072753906
at epoch : 3 train_loss = 1000.2343969726562  valid_loss = 457.67698160807294
at epoch : 4 train_loss = 652.530283203125  valid_loss = 422.07728068033856
at epoch : 5 train_loss = 521.9027917480469  valid_loss = 334.9480489095052


In [14]:
# forth_model trainning
forth_model = GCN_More_layer_model(data_num_features=data.num_features, embedding_size=32)
forth_model.load_state_dict(torch.load(PATH_forth_model_init))
optimizer_forth = torch.optim.Adam(forth_model.parameters(), lr=0.0007)
loss_train_track_forth, loss_valid_track_forth, stop_at_epoch_forth = train(model=forth_model, optimizer=optimizer_forth, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 168668.0931640625  valid_loss = 14573.262451171875
at epoch : 2 train_loss = 14536.6262109375  valid_loss = 9919.899820963541
at epoch : 3 train_loss = 7700.442963867187  valid_loss = 7732.3857421875
at epoch : 4 train_loss = 5078.977646484375  valid_loss = 4768.129150390625
at epoch : 5 train_loss = 3945.779931640625  valid_loss = 4209.951985677083


In [15]:
# fifth_model trainning
fifth_model = GCN_More_layer_model(data_num_features=data.num_features, embedding_size=64)
fifth_model.load_state_dict(torch.load(PATH_fifth_model_init))
optimizer_fifth = torch.optim.Adam(fifth_model.parameters(), lr=0.0007)
loss_train_track_fifth, loss_valid_track_fifth, stop_at_epoch_fifth = train(model=fifth_model, optimizer=optimizer_fifth, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 290317.925  valid_loss = 80146.95833333333
at epoch : 2 train_loss = 47168.927578125  valid_loss = 33557.8056640625
at epoch : 3 train_loss = 26031.3353125  valid_loss = 22366.338704427082
at epoch : 4 train_loss = 18155.06140625  valid_loss = 16714.211100260418
at epoch : 5 train_loss = 14377.90279296875  valid_loss = 15549.471842447916


In [16]:
# sixth_model trainning
sixth_model = GCN_More_layer_model(data_num_features=data.num_features, embedding_size=128)
sixth_model.load_state_dict(torch.load(PATH_sixth_model_init))
optimizer_sixth = torch.optim.Adam(sixth_model.parameters(), lr=0.0007)
loss_train_track_sixth, loss_valid_track_sixth, stop_at_epoch_sixth = train(model=sixth_model, optimizer=optimizer_sixth, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 653378.670625  valid_loss = 169392.13802083334
at epoch : 2 train_loss = 79683.50765625  valid_loss = 75401.412109375
at epoch : 3 train_loss = 46895.375390625  valid_loss = 47709.873697916664
at epoch : 4 train_loss = 34297.579921875  valid_loss = 37001.234049479164
at epoch : 5 train_loss = 26660.2360546875  valid_loss = 28468.814778645832


In [20]:
# seventh_model trainning
seventh_model = GCN_Less_layer_model(data_num_features=data.num_features, embedding_size=32)
seventh_model.load_state_dict(torch.load(PATH_seventh_model_init))
optimizer_seventh = torch.optim.Adam(seventh_model.parameters(), lr=0.0007)
loss_train_track_seventh, loss_valid_track_seventh, stop_at_epoch_seventh = train(model=seventh_model, optimizer=optimizer_seventh, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 457.20014053344727  valid_loss = 54.9495644569397
at epoch : 2 train_loss = 50.36080497741699  valid_loss = 46.65918699900309
at epoch : 3 train_loss = 35.57845058441162  valid_loss = 30.560200055440266
at epoch : 4 train_loss = 26.049870090484617  valid_loss = 23.178395748138428
at epoch : 5 train_loss = 19.84479419708252  valid_loss = 23.055582523345947


In [22]:
# eighth_model trainning
eighth_model = GCN_Less_layer_model(data_num_features=data.num_features, embedding_size=64)
eighth_model.load_state_dict(torch.load(PATH_eighth_model_init))
optimizer_eighth = torch.optim.Adam(eighth_model.parameters(), lr=0.0007)
loss_train_track_eighth, loss_valid_track_eighth, stop_at_epoch_eighth = train(model=eighth_model, optimizer=optimizer_eighth, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 77.4163667678833  valid_loss = 28.690818468729656
at epoch : 2 train_loss = 17.58393970489502  valid_loss = 13.077176411946615
at epoch : 3 train_loss = 11.762789325714111  valid_loss = 11.498063882191977
at epoch : 4 train_loss = 9.885744438171386  valid_loss = 10.528969685236612
at epoch : 5 train_loss = 8.812910270690917  valid_loss = 10.373045921325684


In [23]:
# ninth_model trainning
ninth_model = GCN_Less_layer_model(data_num_features=data.num_features, embedding_size=128)
ninth_model.load_state_dict(torch.load(PATH_ninth_model_init))
optimizer_ninth = torch.optim.Adam(ninth_model.parameters(), lr=0.0007)
loss_train_track_ninth, loss_valid_track_ninth, stop_at_epoch_ninth = train(model=ninth_model, optimizer=optimizer_ninth, loss_fn=loss_fn,
                                                                            epochs=5, is_early_stop=False, show_result_at=1,
                                                                            device=device, validation_set=validation_set, training_set=training_set)


at epoch : 1 train_loss = 1114.9195788574218  valid_loss = 282.3285140991211
at epoch : 2 train_loss = 152.2808544921875  valid_loss = 57.048532803853355
at epoch : 3 train_loss = 39.998948135375976  valid_loss = 27.512378374735516
at epoch : 4 train_loss = 26.421061401367187  valid_loss = 25.96221923828125
at epoch : 5 train_loss = 22.711476974487304  valid_loss = 21.40555222829183


In [53]:
# sklearn.metrics.mean_squared_error(df['y_real'], df['y_pred'])

0.8329761435263996

In [ ]:
# # first model
# embedding_size = 64
# class GCN_first_model(torch.nn.Module):
#     def __init__(self):
#         torch.manual_seed(seed_value)
#         # Init parent
#         super(GCN_first_model, self).__init__()

#         # define layer
#         self.initial_conv = GCNConv(data.num_features, embedding_size)
#         self.conv1 = GCNConv(embedding_size, embedding_size)
#         self.conv2 = GCNConv(embedding_size, embedding_size)
    
#         # define linear layer
#         self.out = Linear(embedding_size*2, 1)
    
#     def init_weights(self):
#         # Use Xavier/Glorot initialization for GCNConv layers
#         for layer in [self.initial_conv, self.conv1, self.conv2]:
#             if isinstance(layer, GCNConv):
#                 # layer.reset_parameters()
#                 if isinstance(layer, GCNConv):
#                     init.normal_(layer.lin.weight)
#                     if layer.bias is not None:
#                         init.zeros_(layer.bias)

#         # Initialize weights for the Linear layer
#         init.xavier_normal_(self.out.weight)
#         if self.out.bias is not None:
#             init.zeros_(self.out.bias)

#     def forward(self, x, edge_index, batch_index):
        
#         # first layer
#         hidden = self.initial_conv(x, edge_index)
#         hidden = F.relu(hidden)
#         # second layer
#         hidden = self.conv1(hidden, edge_index)
#         hidden = F.relu(hidden)
#         # third layer
#         hidden = self.conv2(hidden, edge_index)
#         hidden = F.relu(hidden)

#         # global pooling
#         hidden = torch.cat([gmp(hidden, batch_index),
#                             gap(hidden, batch_index)], dim=1)
                            

#         # apply linear layer
#         out = self.out(hidden)
#         return out
    

# # second model
# embedding_size = 32
# class GCN_second_model(torch.nn.Module):
#     def __init__(self):
#         torch.manual_seed(seed_value)
#         # Init parent
#         super(GCN_second_model, self).__init__()

#         # define layer
#         self.initial_conv = GCNConv(data.num_features, embedding_size)
#         self.conv1 = GCNConv(embedding_size, embedding_size)
#         self.conv2 = GCNConv(embedding_size, embedding_size)
    
#         # define linear layer
#         self.out = Linear(embedding_size*2, 1)

#     def init_weights(self):
#         # Use Xavier/Glorot initialization for GCNConv layers
#         for layer in [self.initial_conv, self.conv1, self.conv2]:
#             if isinstance(layer, GCNConv):
#                 # layer.reset_parameters()
#                 if isinstance(layer, GCNConv):
#                     init.normal_(layer.lin.weight)
#                     if layer.bias is not None:
#                         init.zeros_(layer.bias)

#         # Initialize weights for the Linear layer
#         init.xavier_normal_(self.out.weight)
#         if self.out.bias is not None:
#             init.zeros_(self.out.bias)

#     def forward(self, x, edge_index, batch_index):
        
#         # first layer
#         hidden = self.initial_conv(x, edge_index)
#         hidden = F.relu(hidden)
#         # second layer
#         hidden = self.conv1(hidden, edge_index)
#         hidden = F.relu(hidden)
#         # third layer
#         hidden = self.conv2(hidden, edge_index)
#         hidden = F.relu(hidden)

#         # global pooling
#         hidden = torch.cat([gmp(hidden, batch_index),
#                             gap(hidden, batch_index)], dim=1)

#         # apply linear layer
#         out = self.out(hidden)
#         return out

# # third model
# embedding_size = 128
# class GCN_third_model(torch.nn.Module):
#     def __init__(self):
#         torch.manual_seed(seed_value)
#         # Init parent
#         super(GCN_third_model, self).__init__()

#         # define layer
#         self.initial_conv = GCNConv(data.num_features, embedding_size)
#         self.conv1 = GCNConv(embedding_size, embedding_size)
#         self.conv2 = GCNConv(embedding_size, embedding_size)
    
#         # define linear layer
#         self.out = Linear(embedding_size*2, 1)

#     def init_weights(self):
#         # Use Xavier/Glorot initialization for GCNConv layers
#         for layer in [self.initial_conv, self.conv1, self.conv2]:
#             if isinstance(layer, GCNConv):
#                 # layer.reset_parameters()
#                 if isinstance(layer, GCNConv):
#                     init.normal_(layer.lin.weight)
#                     if layer.bias is not None:
#                         init.zeros_(layer.bias)

#         # Initialize weights for the Linear layer
#         init.xavier_normal_(self.out.weight)
#         if self.out.bias is not None:
#             init.zeros_(self.out.bias)

#     def forward(self, x, edge_index, batch_index):
        
#         # first layer
#         hidden = self.initial_conv(x, edge_index)
#         hidden = F.relu(hidden)
#         # second layer
#         hidden = self.conv1(hidden, edge_index)
#         hidden = F.relu(hidden)
#         # third layer
#         hidden = self.conv2(hidden, edge_index)
#         hidden = F.relu(hidden)

#         # global pooling
#         hidden = torch.cat([gmp(hidden, batch_index),
#                             gap(hidden, batch_index)], dim=1)

#         # apply linear layer
#         out = self.out(hidden)
#         return out
    

# class EarlyStopping:
#     def __init__(self, tolerance=5, min_delta=0):
#         self.tolerance = tolerance
#         self.min_delta = min_delta
#         self.counter = 0
#         self.early_stop = False
#         self.best_validation_loss = float('inf')

#     def __call__(self, train_loss, validation_loss):
#         if train_loss > (self.best_validation_loss - self.min_delta):
#             self.counter = 0
#             self.best_validation_loss = validation_loss

#         elif  self.best_validation_loss - self.min_delta > train_loss  :
#             self.best_validation_loss = validation_loss
#             self.counter += 1
#             print(f'case valid better case {self.counter}')
#             if self.counter >= self.tolerance:
#                 self.early_stop = True

#         else : 
#             print('something wrong')


In [ ]:
# # training function
# def train(model,optimizer,loss_fn,epochs:int=1000, is_early_stop:bool = True,show_result_at:int = 0):
#     earlystop = EarlyStopping(tolerance=7,min_delta=1)
#     train_losses_epoch = []
#     valid_losses_epoch = []
#     stop_at_epoch = epochs
#     for epoch in range(epochs):
#         model = model.to(device)
#         train_losses = []
#         valid_losses = []
        
#         #train step
#         for batch in training_set:
#             # Use GPU
#             batch.to(device)
#             # Reset gradients
#             optimizer.zero_grad()
#             # Passing the node features and the connection info
#             pred  = model(batch.x.float(), batch.edge_index, batch.batch)
#             # Calculating the loss and gradients
#             train_loss = loss_fn(pred, batch.y)
#             train_loss.backward()
#             if torch.cuda.is_available() : float_train_loss = float(train_loss.cpu().detach().numpy().astype(float))
#             else : float_train_loss = float(train_loss.detach().numpy().astype(float))
#             train_losses.append(float_train_loss)
#             # Update using the gradients
#             optimizer.step()
        
#         #valid step
#         model.eval()
#         for batch in validation_set:
#             # Use GPU
#             batch.to(device)
#             # Passing the node features and the connection info
#             pred = model(batch.x.float(), batch.edge_index, batch.batch)
#             # Calculating the loss
#             valid_loss = loss_fn(pred, batch.y)
#             if torch.cuda.is_available(): float_valid_loss = float(valid_loss.cpu().detach().numpy().astype(float))
#             else : float_valid_loss = float(valid_loss.detach().numpy().astype(float))
#             valid_losses.append(float_valid_loss)
        
#         #calculate average loss
#         average_train_loss = sum(train_losses)/len(train_losses)
#         average_valid_loss = sum(valid_losses)/len(valid_losses)
#         train_losses_epoch.append(average_train_loss)
#         valid_losses_epoch.append(average_valid_loss)
#         if (show_result_at != 0) and ((epoch+1)%show_result_at == 0 ): print(f"at epoch : {epoch} train_loss = {average_train_loss}  valid_loss = {average_valid_loss}")
#         if (is_early_stop):earlystop(train_loss=average_train_loss, validation_loss=average_valid_loss)
#         if earlystop.early_stop and is_early_stop:
#             stop_at_epoch = epoch+1
#             print(f'result at {epoch+1} is {earlystop.early_stop}')
#             break
#     # test_result = test(model,loss_fn=loss_fn)
#     return train_losses_epoch, valid_losses_epoch, stop_at_epoch

# def test(model,loss_fn):
#     test_loss_list = []
#     for batch in test_set:
#         batch.to(device)
#         # Passing the node features and the connection info
#         pred = model(batch.x.float(), batch.edge_index, batch.batch)
#         # Calculating the loss
#         test_loss = loss_fn(pred, batch.y)
#         if torch.cuda.is_available(): float_test_loss = float(test_loss.cpu().detach().numpy().astype(float))
#         else : float_test_loss = float(test_loss.detach().numpy().astype(float))
#         test_loss_list.append(float_test_loss)
#     average_train_loss = sum(test_loss_list)/len(test_loss_list)
#     return average_train_loss

# # def grid_search(model, loss_fn, epoch_list, embed_node_list, embed_classifire_layer):

